# 📚 Natural Language Processing (NLP)

En esta práctica vamos a trabajar con herramientas de procesamiento de lenguaje natural utilizando la biblioteca `nltk`. Exploraremos técnicas fundamentales como la tokenización, eliminación de palabras vacías, lematización y modelado de texto.


In [ ]:
# Si usamos colab
from google.colab import drive
drive.mount('/content/drive')

# 🧰 Imports


- `punkt`: para dividir texto en palabras y oraciones.
- `stopwords`: lista de palabras comunes que suelen eliminarse.
- `wordnet`: base de datos léxica utilizada para la lematización.

In [ ]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# 📥 Carga de datos

📌 El dataset contiene reseñas de medicamentos y su respectiva calificación (`rating`), útil para tareas de clasificación de sentimientos o análisis de texto.


In [ ]:
# Si usamos colab
df = pd.read_csv("/content/drive/My Drive/data/new_drug_train.tsv", sep = "\t")

# Si usamos local
# df = pd.read_csv("new_drug_train.tsv", sep = "\t")

In [ ]:
df.info()

In [ ]:
df

# 🧹 Preprocesamiento de Texto

Antes de alimentar un modelo con texto, es fundamental limpiarlo y normalizarlo. Las tareas realizadas incluyen:

- 🔡 Conversión a minúsculas.
- ❌ Eliminación de caracteres no alfabéticos (como puntuación).
- ✂️ Tokenización (dividir texto en palabras).
- 🧽 Eliminación de stopwords (palabras comunes que no aportan mucho significado).
- 🌱 Lematización (reducir palabras a su forma base).

También se remueve la columna `review` original, dejando solo el texto limpio.


In [ ]:
stop_words = set(stopwords.words('english'))
negation_words = {"no", "not", "never", "none", "n't"}
stop_words = stop_words - negation_words
print(len(stop_words))

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['clean_review'] = df['review'].apply(preprocess_text)
df[['review', 'clean_review']].head()

In [ ]:
df = df.drop(columns = ['review'])

# ✂️ División de Datos

Separamos el conjunto de datos en entrenamiento y test utilizando `train_test_split`.

Esto nos permite entrenar el modelo con una parte del dataset y evaluar su desempeño con otra que no ha visto.

In [ ]:
from sklearn.model_selection import train_test_split
random_state = 17

X, y = df.loc[:, df.columns != 'rating'], df["rating"]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=(1.0/3),
    random_state=random_state)

# 🔢 Count Vectorizer

Transformamos el texto en una representación numérica mediante el método `Bag of Words`, usando `CountVectorizer`.

- Cada columna representa una palabra del vocabulario.
- Cada fila representa una review.
- El valor indica cuántas veces aparece una palabra en la review.

> 🔍 Usamos `min_df=0.02` para filtrar palabras poco frecuentes y reducir la dimensionalidad.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Ponemos un min_df = 0.2 para reducir la cantidad de vectores a solo aquellos que se repiten mas del 2%
vectorizer = CountVectorizer(stop_words="english", min_df=0.02)

## 📐 Fit vs Transform

Aplicamos `.fit_transform()` sobre el conjunto de entrenamiento para construir el vocabulario y vectorizar.

Para el conjunto de test, usamos solamente `.transform()` para evitar "mirar" datos del test durante el entrenamiento (data leakage).


Creamos una matriz con los tokens que aparece cada palabra en cada review. Utilizamos *fit* para que aprenda el vocabulario de los textos (identificar las palabras únicas) y *transform* para convertir convertir cada documentos del corpus en una matriz donde las filas representan las reviews y las columnas las palabras.

In [ ]:
X_train_review_tok_matrix = vectorizer.fit_transform(X_train["clean_review"])
X_test_review_tok_matrix = vectorizer.transform(X_test["clean_review"])

## 📊 De matriz dispersa a DataFrame

Convertimos la matriz esparsa resultante en un `DataFrame` de pandas para facilitar su uso junto con las otras variables predictoras.


In [ ]:
X_train_review_tok = pd.DataFrame(X_train_review_tok_matrix.toarray())
X_test_review_tok = pd.DataFrame(X_test_review_tok_matrix.toarray())

## 🔗 Unificación del dataset

Combinamos los vectores numéricos generados a partir del texto con las otras variables del conjunto de datos original.

Esto permite entrenar modelos que consideren tanto texto como otras posibles variables estructuradas.


In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
X_train_review_tok.reset_index(drop=True, inplace=True)
X_test_review_tok.reset_index(drop=True, inplace=True)

In [ ]:
X_train_concat = pd.concat([X_train, X_train_review_tok], axis=1)
X_test_concat = pd.concat([X_test, X_test_review_tok], axis=1)
X_train_concat

## 🌳 Modelo de Clasificación

Entrenaremos un modelo de Árbol de Decisión (`DecisionTreeClassifier`) para predecir la calificación del medicamento (`rating`), usando los datos vectorizados de las reviews.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=10, criterion='gini', random_state=42)

tree.fit(X_train_concat.iloc[:, 6:], y_train)

In [ ]:
from sklearn.metrics import accuracy_score

# Realizar predicciones en los conjuntos de entrenamiento y prueba
y_train_pred = tree.predict(X_train_concat.iloc[:, 6:])
y_test_pred = tree.predict(X_test_concat.iloc[:, 6:])

# Calcular y guardar las precisiones
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Imprimir las precisiones para verificación
print("Train accuracy: ", train_accuracy)
print("Test accuracy: ", test_accuracy)

# 📘 TF-IDF Vectorizer

A diferencia del `CountVectorizer`, que solo cuenta apariciones de palabras, el `TfidfVectorizer` asigna pesos según la importancia de cada palabra en el documento y en el corpus.

- **TF** (*Term Frequency*): Frecuencia de una palabra en el documento.
- **IDF** (*Inverse Document Frequency*): Penaliza palabras comunes en todos los documentos.

> ⚖️ Esto ayuda a reducir el peso de palabras que aparecen en casi todos los textos y resaltar las más informativas.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(stop_words="english", min_df=0.02)

In [ ]:
tfidf_X_train_review_tok_matrix = tf_idf_vectorizer.fit_transform(X_train["clean_review"])
tfidf_X_test_review_tok_matrix = tf_idf_vectorizer.transform(X_test["clean_review"])

In [ ]:
tfidf_X_train_review_tok = pd.DataFrame(tfidf_X_train_review_tok_matrix.toarray())
tfidf_X_test_review_tok = pd.DataFrame(tfidf_X_test_review_tok_matrix.toarray())

In [ ]:
tfidf_X_train_review_tok.reset_index(drop=True, inplace=True)
tfidf_X_test_review_tok.reset_index(drop=True, inplace=True)

In [ ]:
tfidf_X_train_concat = pd.concat([X_train, tfidf_X_train_review_tok], axis=1)
tfidf_X_test_concat = pd.concat([X_test, tfidf_X_test_review_tok], axis=1)
tfidf_X_test_concat

## 🌳 Modelo de Clasificación


In [ ]:
tree = DecisionTreeClassifier(max_depth=10, criterion='gini', random_state=42)

tree.fit(tfidf_X_train_concat.iloc[:, 6:], y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_train_pred_tfidf = tree.predict(tfidf_X_train_concat.iloc[:, 6:])
y_test_pred_tfidf = tree.predict(tfidf_X_test_concat.iloc[:, 6:])

train_accuracy_tfidf = accuracy_score(y_train, y_train_pred_tfidf)
test_accuracy_tfidf = accuracy_score(y_test, y_test_pred_tfidf)

print("Train accuracy (TF-IDF): ", train_accuracy_tfidf)
print("Test accuracy (TF-IDF): ", test_accuracy_tfidf)

# Comparación Métricas

In [ ]:
data = {
    'Métrica': ['Precisión en Entrenamiento', 'Precisión en Prueba'],
    'Modelo con CountVectorizer': [train_accuracy, test_accuracy],
    'Modelo con TF-IDF': [train_accuracy_tfidf, test_accuracy_tfidf]
}

df_comparacion_modelos = pd.DataFrame(data)

df_comparacion_modelos